In [ ]:
import os
import re
import json
from datetime import datetime

In [ ]:
class email():
    def __init__(self, id, message_id, data, subject, sender, receiver, day_week,time, time_zone, date, beginning, farewell, answer, length, relative_path):
        self._id = id
        self._message_id = message_id
        self._data = data
        self._subject = subject
        self._sender = sender
        self._receiver = receiver
        self._number_of_receivers = len(receiver)
        self._day_week = day_week
        self._time = time
        self._time_zone = time_zone
        self._responded = False
        self._responses = set()
        self._date = date
        self._beginning = beginning
        self._farewell = farewell
        self._answer = answer
        self._length = length
        self._relative_path = relative_path

In [ ]:
def email_to_dict(email):
    return email.__dict__

In [ ]:
def read_email(filename, message_ids, id):
    with open(filename, 'r', errors='ignore') as file:
        data = file.read()

        message_id = re.search(r'Message-ID: (.*)',data)
        if message_id:
            message_id = message_id.group(1)
            if message_id in message_ids:
                return None
            

        id = id
        Date = r'Date: (\w+), (\d+ \w+ \d+) (\d+:\d+:\d+) (-\d{4} \(.*?\))'
        coincidencias = re.findall(Date, data)
        for day_week, date, time, time_zone in coincidencias:
            day_week = day_week
            date = date
            time = time
            time_zone = time_zone
            
        subject_match = re.search(r'Subject: (.*)', data)
        if subject_match:
            subject = subject_match.group(1)
        else:
            subject = None
            
        sender_match = re.search(r'\nFrom: (.*)', data)
        if sender_match:
            sender = sender_match.group(1)
        else:
            sender = None
            
        receiver = re.findall(r'\nTo: (.*)',data)
        input_data = str(receiver)[1:-1]
        receiver_list = input_data.split(', ')
        receiver_list = [receiver.strip() for receiver in receiver_list]
        receiver_list = [s.replace("'", "") for s in receiver_list]
        receiver = [s.replace("\"", "'") for s in receiver_list]
        
        beginning_pattern = (r"(?:\b(?:hello|hi|hey|good (?:morning|afternoon|evening)|greetings|dear|my dearest|hi there|hello there|hi all|hi everyone|hello everyone|dearest|yo|what's up|hey pal|yolo|howdy|how's it going|how are you|what's happening|what's new|good to see you|long time no see|it's been a while|what's cracking|how's everything))\b")
        beginning = re.findall(beginning_pattern, data, flags=re.IGNORECASE)
        
        
        farewell_pattern = r"(?:\b(?:Sincerely|Regards|Best regards|Kind regards|Yours sincerely|Yours faithfully|Best wishes|Take care|Warm regards|With gratitude|With best regards|Cheers|Cordially|Respectfully|Until next time|All the best|Have a great day|Talk to you soon|Looking forward to hearing from you|With thanks|Later|Catch you later|Farewell|Peace out|Stay well|All my best|Goodbye|ByeFond regards|Take it easy|So long|See you around|Keep in touch|With warm regards|With all my heart)\b)"
        farewell = re.findall(farewell_pattern, data, flags=re.IGNORECASE)
        
        if re.search(r"-----Original Message-----", data):
            answer = 1
        else:
            answer = 0
        
        
        body_b = data.find("X-FileName:")
        if body_b != -1:
            end_l = data.find("\n", body_b)
            if end_l != -1:
                body = data[end_l + 1:]
                words = body.split()
        length = len(words)

        relative_path = os.path.relpath(filename, start=os.path.join(os.getcwd(), '..'))     

        email_object = email(id, message_id, data, subject, sender, receiver, day_week,time, time_zone, date,beginning,farewell, answer, length, relative_path)
        return email_object
        

In [ ]:
list_of_emails = []
set_of_message_ids = set()
counter = 0

base_dir = os.path.join(os.path.dirname(os.getcwd()), 'maildir/')

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if counter > 0 and counter % 100000 == 0:
            emails_dict_list = [email_to_dict(email) for email in list_of_emails]
            file_path = f'../data/emails{counter // 100000}.json'
            with open(file_path, 'w') as json_file:
                json.dump(emails_dict_list, json_file)
            list_of_emails = []

        file_path = os.path.join(root, file)
        tmp_email = read_email(file_path, set_of_message_ids, counter)
        if tmp_email is not None:
            counter += 1
            list_of_emails.append(tmp_email)
            set_of_message_ids.add(tmp_email._message_id)


In [ ]:
def extract_answer_info(email_content):
    try:
        # Find all occurrences of forwarded messages
        # forwarded_messages = re.findall(r'(-{3,} Forwarded by .+?\n(?:.|\n)*?)\n-{3,}', email_content, re.DOTALL)
        forwarded_messages = re.findall(r'(-{3,} Forwarded by .+?\n(?:.|\n)*?)(?=^-{3,} Original Message -{3,}|$)', email_content, re.DOTALL)
        for forwarded_message in forwarded_messages:
            # Check if the forwarded message does not contain the "--- Original Message ---" delimiter
            if r"-{3,}Original Message-{3,}" not in forwarded_message:
                # Remove the forwarded message from the list
                forwarded_messages.remove(forwarded_message)


        # Extract the last forwarded message
        last_forwarded_message = forwarded_messages[-1] if forwarded_messages else None 

        if last_forwarded_message is None:
            return None, None, None, None   
        
        # Find the sender, date, and subject of the last forwarded message
        email_pattern = r'[\w\.-]+@[\w\.-]+'

        # Find all email addresses in the text
        email_address = re.findall(email_pattern, last_forwarded_message)[0]
        sender = email_address
        
        timestamp_pattern = r'\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2} [AP]M'

        # Search for the email address and timestamp in the text
        timestamp_match = re.search(timestamp_pattern, last_forwarded_message)

        # Extract the timestamp if found
        timestamp = timestamp_match.group() if timestamp_match else None

        # Parse the date string
        parsed_date = datetime.strptime(timestamp, '%m/%d/%Y %I:%M:%S %p')

        # Format the parsed date as 'DD Mon YYYY' (for date) and 'HH:MM:SS' (for time)
        date = parsed_date.strftime('%d %b %Y')
        time = parsed_date.strftime('%I:%M:%S %p')
            
        subject_match = re.search(r'Subject: (.+)', last_forwarded_message)
        subject = subject_match.group(1) if subject_match else None
        
        return sender, date, time, subject
    except:
        return None, None, None, None

In [ ]:
def extract_answered_message(email_content):
    try:
        original_message_match = re.search(r'-{3,}Original Message-{3,}(.+?)(?:=======|$)', email_content, re.DOTALL)
        original_message = original_message_match.group(1).strip() if original_message_match else None
        # Extract sender, date, and subject
        sender_match = re.search(r'From: (.+)', original_message)
        text = sender_match.group(0)

        # Extract the name "Steffes, James" from the text
        name_match = re.search(r'From: \s*(.*)', text)
        if name_match:
            sender = name_match.group(1).rstrip()

        # sender = sender_match.group(1).split(' ')[0] if sender_match else None

        date_match = re.search(r'Sent:( |\t)(.+)', original_message)
        
        if date_match is None:
            date_match = re.search(r'Date: (.+)', original_message)
            date = date_match.group(1) if date_match else None
            parsed_date = datetime.strptime(date, "%a, %d %b %Y %H:%M:%S %z")

            # Extract date and time components
            date = parsed_date.strftime("%d %b %Y")
            time = parsed_date.strftime("%I:%M:%S %p")
        else:
            date = date_match.group(2) if date_match else None
            parsed_date = datetime.strptime(date, "%A, %B %d, %Y %I:%M %p")

            # Format date and time
            date = parsed_date.strftime("%d %b %Y")
            time = parsed_date.strftime("%I:%M:%S %p")


        subject_match = re.search(r'Subject:( |\t)(.+)', original_message)
        subject = subject_match.group(2) if subject_match else None

        return sender, date, time, subject
    except:
        return None, None, None, None

In [ ]:
def find_email(list_of_emails, sender, date, time, subject):
    for email_obj in list_of_emails:
        if (email_obj._date == date and
            email_obj._time == time and
            email_obj._subject == subject):
            return email_obj
    # If no matching email is found
    return None    

In [ ]:
print(len(list_of_emails))

In [ ]:
for index, email in enumerate(list_of_emails):
    if email._answer == 0:
        continue
    parent_sender, parent_date, parent_time, parent_subject = extract_answered_message(email._data)
    answer_sender, answer_date, answer_time, answer_subject = extract_answer_info(email._data)
    if parent_sender is not None:
        parent_email = find_email(list_of_emails, parent_sender, parent_date, parent_time, parent_subject)
        if parent_email:
            print(parent_email)
    if answer_sender is not None:
        print('halko')
        answer_email = find_email(list_of_emails, answer_sender, answer_date, answer_time, answer_subject)
    if parent_email:
        parent_email._responded = True
        parent_email._responses.append(answer_email._id)

In [ ]:
print(len(list_of_emails))
print(len(set_of_message_ids))

In [ ]:
# emails_dict_list = [email_to_dict(email) for email in list_of_emails]

# file_path = f'../data/emails{counter // 100000 + 1}.json'

# with open(file_path, 'w') as json_file:
#     json.dump(emails_dict_list, json_file)